In [1]:
!pip install pandas numpy faker


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   --------------------- ------------------ 1.0/2.0 MB 2.7 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 2.2 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 2.2 MB/s  0:00:01


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import pandas as pd
import numpy as np
from faker import Faker
import uuid
from datetime import datetime, timedelta
import random

fake = Faker()
np.random.seed(42)
random.seed(42)

# -----------------------------
# CONFIG (FAANG SCALE)
# -----------------------------
NUM_USERS = 10000
DAYS = 60
SESSIONS_PER_USER = (10, 16)  # min, max

# Funnel probabilities
P_VIEW = 0.95
P_ADD_CART = 0.55
P_CHECKOUT = 0.80
P_PURCHASE_CONTROL = 0.55
P_PURCHASE_VARIANT = 0.70

# -----------------------------
# USERS TABLE
# -----------------------------
users = []

for _ in range(NUM_USERS):
    users.append({
        "user_id": str(uuid.uuid4()),
        "signup_date": fake.date_between(start_date='-90d', end_date='-1d'),
        "country": random.choice(["India", "US", "UK", "Germany"]),
        "device_type": random.choice(["mobile", "desktop"]),
        "is_new_user": random.choice([True, False])
    })

users_df = pd.DataFrame(users)

# -----------------------------
# EXPERIMENTS TABLE
# -----------------------------
experiments_df = users_df[["user_id"]].copy()
experiments_df["experiment_name"] = "checkout_redesign"
experiments_df["variant"] = np.where(
    np.random.rand(NUM_USERS) < 0.5, "control", "variant"
)
experiments_df["assignment_date"] = fake.date_between(start_date='-30d', end_date='today')

# -----------------------------
# EVENTS TABLE (HEAVY)
# -----------------------------
events = []
start_date = datetime.now() - timedelta(days=DAYS)

for _, user in users_df.iterrows():
    num_sessions = random.randint(*SESSIONS_PER_USER)

    for _ in range(num_sessions):
        session_id = str(uuid.uuid4())
        base_time = start_date + timedelta(
            minutes=random.randint(0, DAYS * 24 * 60)
        )

        # Page views (1–4 per session)
        for i in range(random.randint(3, 10)):
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user.user_id,
                "event_type": "page_view",
                "event_time": base_time + timedelta(seconds=i * 5),
                "product_id": None,
                "session_id": session_id
            })

        if random.random() < P_VIEW:
            for i in range(random.randint(2, 6)):
                events.append({
                    "event_id": str(uuid.uuid4()),
                    "user_id": user.user_id,
                    "event_type": "view_product",
                    "event_time": base_time + timedelta(seconds=20 + i * 5),
                    "product_id": f"P{random.randint(1, 200)}",
                    "session_id": session_id
                })

            if random.random() < P_ADD_CART:
                events.append({
                    "event_id": str(uuid.uuid4()),
                    "user_id": user.user_id,
                    "event_type": "add_to_cart",
                    "event_time": base_time + timedelta(seconds=45),
                    "product_id": f"P{random.randint(1, 200)}",
                    "session_id": session_id
                })

                if random.random() < P_CHECKOUT:
                    events.append({
                        "event_id": str(uuid.uuid4()),
                        "user_id": user.user_id,
                        "event_type": "checkout",
                        "event_time": base_time + timedelta(seconds=70),
                        "product_id": None,
                        "session_id": session_id
                    })

events_df = pd.DataFrame(events)

# -----------------------------
# ORDERS TABLE
# -----------------------------
orders = []
experiment_map = dict(zip(experiments_df.user_id, experiments_df.variant))

for _, row in events_df[events_df.event_type == "checkout"].iterrows():
    variant = experiment_map[row.user_id]
    prob = P_PURCHASE_VARIANT if variant == "variant" else P_PURCHASE_CONTROL

    if random.random() < prob:
        orders.append({
            "order_id": str(uuid.uuid4()),
            "user_id": row.user_id,
            "order_time": row.event_time + timedelta(seconds=15),
            "revenue": round(random.uniform(25, 600), 2),
            "payment_status": "success"
        })

orders_df = pd.DataFrame(orders)

# -----------------------------
# EXPORT
# -----------------------------
users_df.to_csv("users.csv", index=False)
experiments_df.to_csv("experiments.csv", index=False)
events_df.to_csv("events.csv", index=False)
orders_df.to_csv("orders.csv", index=False)

print("✅ Data generation complete")
print("Users:", len(users_df))
print("Events:", len(events_df))
print("Orders:", len(orders_df))


✅ Data generation complete
Users: 10000
Events: 1464736
Orders: 33621


In [8]:
import shutil

files = ["users.csv", "events.csv", "experiments.csv", "orders.csv"]

for f in files:
    shutil.move(f, f"data/raw/{f}")


In [9]:
os.listdir("data/raw")


['events.csv', 'experiments.csv', 'orders.csv', 'users.csv']

In [10]:
%pip install psycopg2-binary


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import psycopg2
print(psycopg2.__version__)


2.9.11 (dt dec pq3 ext lo64)


# 1. Connect Jupyter to PostgreSQL

Establish a connection from Jupyter Notebook to a PostgreSQL database using SQLAlchemy and psycopg2.  
This allows executing SQL queries directly from Python for analysis.

In [12]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

username = "postgres"
password = "sumit@123"
host = "localhost"
port = "5432"
database = "product_analytics"

encoded_password = quote_plus(password)

engine = create_engine(
    f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
)

# Example load
users_df = pd.read_csv("data/raw/users.csv")
users_df.to_sql("users", engine, if_exists="replace", index=False)


1000

In [13]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

username = "postgres"
password = "sumit@123"
host = "localhost"
port = "5432"
database = "product_analytics"

encoded_password = quote_plus(password)

engine = create_engine(
    f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
)

# Example load
users_df = pd.read_csv("data/raw/orders.csv")
users_df.to_sql("orders", engine, if_exists="replace", index=False)


621

In [14]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

username = "postgres"
password = "sumit@123"
host = "localhost"
port = "5432"
database = "product_analytics"

encoded_password = quote_plus(password)

engine = create_engine(
    f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
)

# Example load
users_df = pd.read_csv("data/raw/events.csv")
users_df.to_sql("events", engine, if_exists="replace", index=False)


736

In [15]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

username = "postgres"
password = "sumit@123"
host = "localhost"
port = "5432"
database = "product_analytics"

encoded_password = quote_plus(password)

engine = create_engine(
    f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
)

# Example load
users_df = pd.read_csv("data/raw/experiments.csv")
users_df.to_sql("experiments", engine, if_exists="replace", index=False)


1000

In [16]:
pd.read_sql("SELECT COUNT(*) FROM events;", engine)


,count
0,1464736


In [17]:
query = """SELECT
    e.variant,
    COUNT(DISTINCT e.user_id) AS users,
    COUNT(DISTINCT o.user_id) AS purchasers,
    ROUND(
        COUNT(DISTINCT o.user_id)*100.0 /
        COUNT(DISTINCT e.user_id), 2
    ) AS conversion_rate
FROM experiments e
LEFT JOIN orders o ON e.user_id = o.user_id
GROUP BY e.variant;"""
df = pd.read_sql(query, engine)
df


,variant,users,purchasers,conversion_rate
0,control,5076,4888,96.30
1,variant,4924,4862,98.74


In [19]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd

username = "postgres"
password = "sumit@123"
host = "localhost"
port = "5432"
database = "product_analytics"

encoded_password = quote_plus(password)

engine = create_engine(
    f"postgresql+psycopg2://{username}:{encoded_password}@{host}:{port}/{database}"
)


In [20]:
pd.read_sql("SELECT COUNT(*) FROM events;", engine)


,count
0,1464736


# Fetch A/B Test Aggregates from SQL

Query the PostgreSQL database to retrieve aggregated metrics  
such as total users and purchasers for Control and Variant groups.


In [21]:
query = """
SELECT
    e.variant,
    COUNT(DISTINCT e.user_id) AS users,
    COUNT(DISTINCT o.user_id) AS purchasers
FROM experiments e
LEFT JOIN orders o ON e.user_id = o.user_id
GROUP BY e.variant;
"""
df = pd.read_sql(query, engine)
df


,variant,users,purchasers
0,control,5076,4888
1,variant,4924,4862


# Prepare Contingency Table

Construct a 2×2 contingency table representing  
successes (purchases) and failures (non-purchases)  
for Control and Variant groups.

In [22]:
control_users = df.loc[df.variant == "control", "users"].values[0]
control_purchases = df.loc[df.variant == "control", "purchasers"].values[0]

variant_users = df.loc[df.variant == "variant", "users"].values[0]
variant_purchases = df.loc[df.variant == "variant", "purchasers"].values[0]

contingency_table = [
    [control_purchases, control_users - control_purchases],
    [variant_purchases, variant_users - variant_purchases]
]

contingency_table


[[np.int64(4888), np.int64(188)], [np.int64(4862), np.int64(62)]]

# Perform Chi-Square Test

Apply the Chi-Square Test of Independence to determine  
whether the observed difference in conversion rates  
between Control and Variant is statistically significant.


In [23]:
from scipy.stats import chi2_contingency

chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Chi-square:", chi2)
print("p-value:", p_value)


Chi-square: 60.278295928260505
p-value: 8.235045256018143e-15


# Interpret Statistical Result



In [24]:
alpha = 0.05

if p_value < alpha:
    print("✅ Statistically significant — Ship the variant")
else:
    print("❌ Not significant — Do NOT ship")


✅ Statistically significant — Ship the variant


# Calculate Conversion Rates and Lift

In [25]:
control_cr = control_purchases / control_users
variant_cr = variant_purchases / variant_users

lift = (variant_cr - control_cr) / control_cr * 100

print(f"Control CR: {control_cr:.2%}")
print(f"Variant CR: {variant_cr:.2%}")
print(f"Lift: {lift:.2f}%")


Control CR: 96.30%
Variant CR: 98.74%
Lift: 2.54%


# Confidence Interval for Conversion Lift


In [26]:
import numpy as np
from statsmodels.stats.proportion import proportion_confint

ci_control = proportion_confint(
    control_purchases, control_users, alpha=0.05, method="wilson"
)

ci_variant = proportion_confint(
    variant_purchases, variant_users, alpha=0.05, method="wilson"
)

print("Control CI:", ci_control)
print("Variant CI:", ci_variant)


Control CI: (0.9574077454636901, 0.9678179802012058)
Variant CI: (0.9838923693801012, 0.9901649415360481)
